In [1]:
import os
import tensorflow as tf
import pickle
import numpy as np
import math as math
import matplotlib.pyplot as plt

C:\tools\Anaconda3\envs\tf\lib\site-packages\h5py\__init__.py:72: UserWarning: h5py is running against HDF5 1.10.2 when it was built against 1.10.3, this may cause problems
  '{0}.{1}.{2}'.format(*version.hdf5_built_version_tuple)


In [ ]:
def vgg16_model(IMAGE_SHAPE=(32, 32, 1), learning_rate=0.001):

    # model = tf.keras.Sequential()

    # # Remove the prediction layer and add to new model
    # for layer in vgg16_model.layers[:-1]: 
    #     model.add(layer)    

    # # Freeze the layers 
    # for layer in model.layers:
    #     layer.trainable = False

    # # Add 'softmax' instead of earlier 'prediction' layer.
    # model.add(tf.keras.layers.Dense(5, activation='softmax'))


    # reset_keras()

    vgg16 = tf.keras.applications.VGG16(include_top=True,
                                        weights='imagenet',
                                        input_tensor=None,
                                        input_shape=(32, 32, 1),
                                        pooling=None,
                                        classes=43)

    # input_ = tf.keras.layers.Input(shape=IMAGE_SHAPE)
    vgg16.compile(loss=tf.keras.losses.sparse_categorical_crossentropy, 
                optimizer=tf.keras.optimizers.Adam(learning_rate), 
                metrics=['accuracy'])

    vgg16.summary()

    return vgg16

In [ ]:
def generate_plots(plot_cntr, history, plot_acc, plot_loss, ep, bs, lr):
    """
    A method that takes the model history of a trained model and plots its:
    1. Training accuracy
    2. Training loss
    3. Validation accuracy
    4. Validation loss
    """
    acc = model_history.history['acc']
    val_acc = model_history.history['val_acc']
    loss = model_history.history['loss']
    val_loss = model_history.history['val_loss']


    plt.figure(plot_cntr)
    plt.suptitle('Accuracy learning curve', fontsize=20)
    plt.xlabel('epochs', fontsize=14)
    plt.ylabel('accuracy', fontsize=14)
    plt.plot(acc, label='training accuracy')
    plt.plot(val_acc, label='validation accuracy')
    plt.xticks(np.arange(0, epochs + epochs/10, epochs/10))
    plt.legend(loc="lower right")
    plt.savefig("{}accuracy_{}_{}_{}.png".format(plot_acc, ep, bs, lr), dpi=250)

    plt.figure(plot_cntr+500)
    plt.suptitle('Loss learning curve', fontsize=20)
    plt.xlabel('epochs', fontsize=14)
    plt.ylabel('loss', fontsize=14)
    plt.plot(loss, label='training loss')
    plt.plot(val_loss, label='validation loss')
    plt.xticks(np.arange(0, epochs + epochs/10, epochs/10))
    plt.legend(loc="upper right")
    plt.savefig("{}loss_{}_{}_{}.png".format(plot_loss, ep, bs, lr), dpi=250)


In [ ]:
def best_results(file, model_history):
    """
    A method that prints the best validation accuracy and loss to a best_results.txt file.
    """
    acc = model_history.history['acc']
    val_acc = model_history.history['val_acc']
    loss = model_history.history['loss']
    val_loss = model_history.history['val_loss']

    
    f = open(file, "w+")
    f.write("Training accuracy = {}\n".format(acc))
    f.write("----------------------\n")
    f.write("Training loss = {}\n".format(loss))
    f.write("----------------------\n")
    f.write("Validation accuracy = {}\n".format(val_acc))
    f.write("----------------------\n")
    f.write("Validation loss = {}\n".format(val_loss))
    f.write("----------------------\n")
    f.write("----------------------\n\n")
    f.close()


In [8]:
# change this for different files
file = 0

In [9]:
# Opening file for reading in binary mode
with open('./data/data{}.pickle'.format(file), 'rb') as f:
    data = pickle.load(f, encoding='latin1')  # dictionary type

In [10]:
# Making channels come at the end
X_train = data['x_train'] = data['x_train'].transpose(0, 2, 3, 1)
X_val = data['x_validation'] = data['x_validation'].transpose(0, 2, 3, 1)
X_test = data['x_test'] = data['x_test'].transpose(0, 2, 3, 1)

y_train = data['y_train']
y_val = data['y_validation']
y_test = data['y_test']

In [11]:
# # Showing loaded data from file
# for i, j in data.items():
#     if i == 'labels':
#         print(i + ':', len(j))
#     else:
#         print(i + ':', j.shape)

x_test: (12630, 32, 32, 3)
y_validation: (4410,)
x_validation: (4410, 32, 32, 3)
labels: 43
x_train: (86989, 32, 32, 3)
y_test: (12630,)
y_train: (86989,)


In [ ]:
epochs = [30]
batch_size = [128, 256, 512]
learning_rate = [0.01, 0.001, 0.0001]

# plot directory for accuracy and loss
results = "./vgg/"
plot_acc = "./vgg/acc/"
plot_loss = "./vgg/loss/"

os.makedirs(plot_acc, exist_ok=True)
os.makedirs(plot_loss, exist_ok=True)

In [ ]:
plot_cntr = 0

with tf.device('/device:GPU:0'):
    
    for ep in epochs:
        for bs in batch_size:
            for lr in learning_rate:
                
                print("-------------------")
                print("Compiling VGG16 model with {} epochs, {} batch size, learning rate {}".format(ep, bs, lr))
                print("-------------------")
                
                model = vgg16_model(learning_rate=lr)
                
                print("-------------------")
                print("Training VGG16 model with {} epochs, {} batch size, learning rate {}".format(ep, bs, lr))
                print("-------------------")
                
                
                history = model.fit(X_train, y_train, batch_size=bs, epochs=ep, validation_data=(X_val, y_val))
                
                predictions = model.predict(X_test, batch_size=bs)
                
                generate_plots(plot_cntr, history, plot_acc, plot_loss, ep, bs, lr)

                best_results(file='{}results_{}_{}_{}.txt'.format(results, ep, bs, lr), history)
                
                plot_cntr+=1